In [18]:
import numpy as np
import pandas as pd

In [19]:
data_frame=pd.read_csv('face.csv')
data_frame=data_frame.iloc[:,2:130]
tf1=data_frame.iloc[0:10,:]
tf2=data_frame.iloc[400:410,:]
cf1=data_frame.iloc[10:400,:]
cf2=data_frame.iloc[410:800,:]
frame=pd.concat([cf1,cf2])

In [20]:
def mean(df):
     mean=df.mean().values
     return np.asarray(mean)

In [21]:
def get_cov(df):
     return np.asarray(df.cov().values)

In [22]:
u=mean(frame)
Σ=get_cov(frame)

In [23]:
λ,e=np.linalg.eig(Σ)
indices=np.argsort(-1*λ)
req_e=np.empty((122,128))
for j in range(122):
     req_e[j]=e[indices[j]]

In [24]:
def get_reduced_frame(df,n):
     a=[]
     reduced_df=pd.DataFrame()
     for j in range(122):
          for i in range(n):
               X=np.asarray(df.iloc[i,:])
               X_u=X-u
               a.append(req_e[j]@X_u.T)
          new_df=pd.DataFrame(a)
          a.clear()
          reduced_df=pd.concat([reduced_df,new_df],axis=1)
     return reduced_df

In [25]:
class_frame1=get_reduced_frame(cf1,390)
class_frame2=get_reduced_frame(cf2,390)
test_frame1=get_reduced_frame(tf1,10)
test_frame2=get_reduced_frame(tf2,10)
test_frame=pd.concat([test_frame1,test_frame2])

In [26]:
u1=mean(class_frame1)
u2=mean(class_frame2)

In [27]:
Σ1=get_cov(class_frame1)
Σ2=get_cov(class_frame2)
if Σ1.all()==Σ2.all():
     print("Σ1 = Σ2")

Σ1 = Σ2


In [28]:
def get_g(X,u,Σ):
    Σinv=np.linalg.inv(Σ)
    X_minus_uT=X-u
    X_minus_u=np.transpose(X_minus_uT)
    return (-1/2)*np.dot(X_minus_uT,np.dot(Σinv,X_minus_u)) #Σ1=Σ2, Pw1=Pw2

In [29]:
def classify(g1,g2):
     if g1>g2:
          return 'male',0
     elif g2>g1:
          return 'female',1
     else:
          return 'none',-1

In [30]:
error1=0
error2=0
for i in range(20):
     X=np.asarray(test_frame.iloc[i,:])
     g1=get_g(X,u1,Σ1)
     g2=get_g(X,u2,Σ2)
     res,j=classify(g1,g2)
     if i in range(0,9):
          if(j!=0): 
               error1+=1
     if i in range(10,19):
          if(j!=1): 
               error2+=1
     print("Test vector",i+1,"is classified into the class",res)
accuracy1=(1-error1/10)*100
print("The accuracy of classification for class male is",accuracy1,"%")
accuracy2=(1-error2/10)*100
print("The accuracy of classification for class female is",accuracy2,"%")
accuracy=(1-(error1+error2)/20)*100
print("The accuracy of classification is",accuracy,"%")
confusion_mat=np.array([[10-error1,error1],[error2,10-error2]])
print(confusion_mat)

Test vector 1 is classified into the class male
Test vector 2 is classified into the class male
Test vector 3 is classified into the class male
Test vector 4 is classified into the class male
Test vector 5 is classified into the class male
Test vector 6 is classified into the class male
Test vector 7 is classified into the class male
Test vector 8 is classified into the class female
Test vector 9 is classified into the class male
Test vector 10 is classified into the class male
Test vector 11 is classified into the class male
Test vector 12 is classified into the class female
Test vector 13 is classified into the class female
Test vector 14 is classified into the class female
Test vector 15 is classified into the class female
Test vector 16 is classified into the class female
Test vector 17 is classified into the class female
Test vector 18 is classified into the class female
Test vector 19 is classified into the class female
Test vector 20 is classified into the class female
The accur